# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [17]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [18]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [19]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [20]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [21]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
try:
    cluster = Cluster(['127.0.0.1'])

    # To establish connection and begin executing queries, need a session
    session = cluster.connect()
except Exception as e:
    print(e)

#### Create Keyspace

In [22]:
# TO-DO: Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)
    

#### Set Keyspace

In [23]:
# TO-DO: Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace("udacity")
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




artist
firstName of user
gender of user
item number in session
last name of user
length of the song
level (paid or free song)
location of the user
sessionId
song title
userId

# Query 1

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

### song_details: 

The sessionId was used as a partition key because the queries will filter by this column.
<br>The itemInSession were used as clustering column to help make up a unique key.

In [24]:
## Table to show artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
query = "CREATE TABLE IF NOT EXISTS song_details "
query = query + "(session_id int,\
                 item_in_session int,\
                 artist text,\
                song_title text,\
                length float,\
                PRIMARY KEY (session_id,item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)                   

In [25]:
## Here we load the file and extract the columns we want to insert them into our table


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert data into the table
        query = "INSERT INTO song_details(session_id,item_in_session,artist, song_title,length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        ## Here we insert the index of the column we want to extracted from file
        session.execute(query, (int(line[8]),int(line[3]),line[0], line[9],float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [26]:
## Here we want to show songs and artist name for session id = 338 and item in session = 4
query = "SELECT artist, song_title,length from song_details WHERE session_id = 338 AND item_in_session = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist, row.song_title, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

# Query 2

## Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

### artist_song_details_for_user:
the userId and sessionId was used as a composite partition key because the queries will filter by these columns. 
<br>The itemInSession were used as clustering column to help make up a unique key.

In [27]:
## Table has song details and user details to show result for user has id = 10 and session = 182 
query = "CREATE TABLE IF NOT EXISTS artist_song_details_for_user"
query = query + "(user_id int,\
                session_id int,\
                item_in_session int,\
                artist text,\
                song_title text,\
                first_name text,\
                last_name text,\
                PRIMARY KEY ((user_id, session_id), item_in_session))"
try:
    session.execute(query)
except Exception as e:
    print(e)            

In [28]:
## Here we load the file and extract the columns we want to insert them into our table


file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert data into the table
        query = "INSERT INTO artist_song_details_for_user(user_id,session_id,item_in_session,artist,song_title,first_name,last_name)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        ## Here we insert the index of the column we want to extracted from file
        session.execute(query, (int(line[10]),int(line[8]),int(line[3]),line[0], line[9],line[1],line[2]))

In [29]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table /// userid = 10, sessionid = 182
### Here we want to show artist name ,song for user has an id = 10
query = "SELECT user_id,artist, song_title,first_name,last_name FROM artist_song_details_for_user WHERE user_id = 10 AND session_id = 182"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_id,row.artist, row.song_title, row.first_name, row.last_name)

10 Down To The Bone Keep On Keepin' On Sylvie F
10 Three Drives Greece 2000 Sylvie F
10 Sebastien Tellier Kilometer Sylvie F
10 Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie F


# Query 3

## Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

### all_users_listened_song: 
the song_title was used as a partition key because the queries will filter by this column. 
<br>The userId were used as clustering column to help make up a unique key.

In [30]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = "CREATE TABLE IF NOT EXISTS all_users_listened_song"
query = query + "(song_title text,\
                user_id int,\
                first_name text,\
                last_name text,\
                PRIMARY KEY ((song_title),user_id))"
try:
    session.execute(query)
except Exception as e:
    print(e)                

In [32]:
## Here we load the file and extract the columns we want to insert them into our table

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
## Insert data into the table
        query = "INSERT INTO all_users_listened_song(song_title,user_id,first_name,last_name)"
        query = query + "VALUES (%s,%s, %s, %s)"
        ## Here we insert the index of the column we want to extracted from file
        session.execute(query, (line[9],int(line[10]),line[1],line[4]))

In [33]:
### Here we select user id,first and last name for each user who listened to the song "All Hands Against His Own"
query = "SELECT song_title,user_id,first_name,last_name FROM all_users_listened_song WHERE song_title= 'All Hands Against His Own' ALLOW FILTERING"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_id,row.first_name, row.last_name,row.song_title)

95 Sara Johnson All Hands Against His Own
80 Tegan Levine All Hands Against His Own
29 Jacqueline Lynch All Hands Against His Own


### Drop the tables before closing out the sessions

In [34]:
## TO-DO: Drop the table before closing out the sessions

#### Drop Table 1 

In [35]:
query = "drop table if exists song_details"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Drop Table 2 

In [36]:
query = "drop table if exists artist_song_details_for_user"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

#### Drop Table 3

In [37]:
query = "drop table if exists all_users_listened_song"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [38]:
session.shutdown()
cluster.shutdown()